In [ ]:
from qTools import *
import Cahit as ck

import scipy.sparse as sp
import numpy as np

from functools import partial
from multiprocessing import Pool, cpu_count

import matplotlib.pyplot as plt
import datetime

In [ ]:
resonatorDimension = 200
g = 1.79
qfreq = 0
resFreq = 2
irreg = False

In [ ]:
qSim = Simulation()
JCSys = qSim.qSys
qSim.delState = True

In [ ]:
cav = Cavity(dimension=resonatorDimension, frequency=resFreq)
JCSys.addSubSys(cav)

qub = JCSys.createSubSys(subClass=Qubit)
qub.frequency = qfreq

In [ ]:
digitalRabi = qProtocol(superSys=JCSys)
JCunitary = freeEvolution(ratio=0.5, superSys=JCSys)
bitFlip = xGate(implementation='instant', superSys=qub)
digitalRabi.addStep(JCunitary, bitFlip, JCunitary, JCunitary, bitFlip, JCunitary)

In [ ]:
JCcoupling = JCSys.JC(cav, qub, g)

In [ ]:
JCSys.initialState = [0,0]

cavFreqSweep = qSim.Loop.addSweep(sys=cav, sweepKey='frequency')
cavFreqSweep.sweepList = np.arange(-4, 4+0.05, 0.05)

"""gSweep = qSim.beforeLoop.addSweep(sys=JCcoupling, sweepKey='couplingStrength')
gSweep.sweepList = np.arange(0, 2, 0.05)"""
qSim.finalTime = 1.2
qSim.stepSize = 0.02

In [ ]:
JCSys.constructCompSys()
cavParity = parityEXP(cav.freeMat)
qubPop = qub.freeMat
def expect(qSim, state):
    qSim.qRes.result = expectationKet(cavParity, state)
    qSim.qRes.result = fidelityKet(qSim.qSys.initialState, state)

qSim.compute = expect

In [ ]:
p = Pool(processes=cpu_count())
print('simulating')

nw = datetime.datetime.now()

results = qSim.run(p)

en = datetime.datetime.now()
print(en-nw)

In [ ]:
fig = plt.figure(figsize=(12,9))
setC = [0.88, 0.25, 0.02, 0.6]
axGrids = ck.Plotting.Functions.grid(2, 1, fig=fig)

ck.Plotting.SimplePlots.colorPlot(cavFreqSweep.sweepList, [x*qSim.stepSize for x in range(qSim.steps+1)], results.results[0],ax=axGrids[0], gif=fig, setC=setC)
ck.Plotting.SimplePlots.colorPlot(cavFreqSweep.sweepList, [x*qSim.stepSize for x in range(qSim.steps+1)], results.results[1],ax=axGrids[1], gif=fig, setC=setC)

In [ ]:
JCSys.reset()
JCSys.createSysCoupling([qub,cav],[QuantumToolbox.operators.sigmax, QuantumToolbox.operators.create], couplingStrength=g)
JCSys.createSysCoupling([cav, qub],[QuantumToolbox.operators.destroy, QuantumToolbox.operators.sigmax], couplingStrength=g)
JCSys.couplingName = 'Rabi'

print(JCSys.couplingName)

In [ ]:
print('simulating')
statesDigit = qSim.run(p)
print(JCSys._QuantumSystem__kept)

In [ ]:
fig = plt.figure(figsize=(12,9))
setC = [0.88, 0.25, 0.02, 0.6]
axGrids = ck.Plotting.Functions.grid(2, 1, fig=fig)

ck.Plotting.SimplePlots.colorPlot(cavFreqSweep.sweepList, [x*qSim.stepSize for x in range(qSim.steps+1)], qSim.qRes.results[0],ax=axGrids[0], gif=fig, setC=setC)
ck.Plotting.SimplePlots.colorPlot(cavFreqSweep.sweepList, [x*qSim.stepSize for x in range(qSim.steps+1)], qSim.qRes.results[1],ax=axGrids[1], gif=fig, setC=setC)

In [ ]:
JCSys.reset(to='JC')
#JCSys.addCoupling([qub,cav],[qtbox.operators.destroy, qtbox.operators.create], g)
#JCSys.addCoupling([cav, qub],[qtbox.operators.destroy, qtbox.operators.create], g)
JCSys.unitary = digitalRabi

In [ ]:
print('simulating')
#p = Pool(processes=cpu_count())
cavFreqSweep.sweepList = np.arange(-2, 2+0.025, 0.025)
'''for i in range(len(cavFreqSweep.sweepList)):
    qSim.evolveTimeIndep(cav, cavFreqSweep.sweepList[i])'''
results = qSim.run(p)
print(JCSys._QuantumSystem__kept)

In [ ]:
fig = plt.figure(figsize=(12,9))
setC = [0.88, 0.25, 0.02, 0.6]
axGrids = ck.Plotting.Functions.grid(2, 1, fig=fig)

ck.Plotting.SimplePlots.colorPlot(cavFreqSweep.sweepList*2, [x*qSim.stepSize for x in range(qSim.steps+1)], qSim.qRes.results[0],ax=axGrids[0], gif=fig, setC=setC)
ck.Plotting.SimplePlots.colorPlot(cavFreqSweep.sweepList*2, [x*qSim.stepSize for x in range(qSim.steps+1)], qSim.qRes.results[1],ax=axGrids[1], gif=fig, setC=setC)
axGrids[0].set_xlim([-4, 4])
axGrids[1].set_xlim([-4, 4])

In [ ]:
JCSys.reset(to='Rabi')
print('simulating')
statesDigit = qSim.run(p)
parityDigit = p.map(partial(expectationKetList, cavParity),statesDigit)
fig = plt.figure(figsize=(12,9))fig = plt.figure(figsize=(12,9))
setC = [0.88, 0.25, 0.02, 0.6]
axGrids = ck.Plotting.Functions.grid(2, 1, fig=fig)

ck.Plotting.SimplePlots.colorPlot(cavFreqSweep.sweepList, [x*qSim.stepSize for x in range(qSim.steps+1)], qSim.qRes.results[0],ax=axGrids[0], gif=fig, setC=setC)
ck.Plotting.SimplePlots.colorPlot(cavFreqSweep.sweepList, [x*qSim.stepSize for x in range(qSim.steps+1)], qSim.qRes.results[1],ax=axGrids[1], gif=fig, setC=setC)
print(JCSys._QuantumSystem__kept)

In [ ]:
JCSys.reset(to='JC')
JCcoupling.couplingStrength = 1
print('simulating')
statesDigit = qSim.run(p)
parityDigit = p.map(partial(expectationKetList, cavParity),statesDigit)
fig = plt.figure(figsize=(12,9))
setC = [0.88, 0.25, 0.02, 0.6]
axGrids = ck.Plotting.Functions.grid(2, 1, fig=fig)

ck.Plotting.SimplePlots.colorPlot(cavFreqSweep.sweepList, [x*qSim.stepSize for x in range(qSim.steps+1)], qSim.qRes.results[0],ax=axGrids[0], gif=fig, setC=setC)
ck.Plotting.SimplePlots.colorPlot(cavFreqSweep.sweepList, [x*qSim.stepSize for x in range(qSim.steps+1)], qSim.qRes.results[1],ax=axGrids[1], gif=fig, setC=setC)
print(JCSys._QuantumSystem__kept)

In [ ]:
JCSys.reset(to=2)
print(JCSys.couplingName)
JCcoupling.couplingStrength = 1.79
print('simulating')
statesDigit = qSim.run(p)
parityDigit = p.map(partial(expectationKetList, cavParity),statesDigit)
fig = plt.figure(figsize=(12,9))
setC = [0.88, 0.25, 0.02, 0.6]
axGrids = ck.Plotting.Functions.grid(2, 1, fig=fig)

ck.Plotting.SimplePlots.colorPlot(cavFreqSweep.sweepList, [x*qSim.stepSize for x in range(qSim.steps+1)], qSim.qRes.results[0],ax=axGrids[0], gif=fig, setC=setC)
ck.Plotting.SimplePlots.colorPlot(cavFreqSweep.sweepList, [x*qSim.stepSize for x in range(qSim.steps+1)], qSim.qRes.results[1],ax=axGrids[1], gif=fig, setC=setC)
print(JCSys._QuantumSystem__kept)